## Install requirements 

In [1]:
    import os
    os.makedirs("/usr/local/lib/python3.11/dist-packages/release_data/ccd_cache", exist_ok=True)
    
    source_ccd_file = "/kaggle/input/protenix-public/af3-dev/release_data/ccd_cache/components.cif"
    target_ccd_file = "/usr/local/lib/python3.11/dist-packages/release_data/ccd_cache/components.cif"
    
    source_rdkit_file = "/kaggle/input/protenix-public/af3-dev/release_data/ccd_cache/components.cif.rdkit_mol.pkl"
    target_rdkit_file = "/usr/local/lib/python3.11/dist-packages/release_data/ccd_cache/components.cif.rdkit_mol.pkl"
    
    # Create the symlinks if the source files exist
    if os.path.exists(source_ccd_file) and not os.path.exists(target_ccd_file):
        try:
            os.symlink(source_ccd_file, target_ccd_file)
            print(f"Created symlink for CCD file")
        except Exception as e:
            print(f"Error creating symlink for CCD file: {e}")
    
    if os.path.exists(source_rdkit_file) and not os.path.exists(target_rdkit_file):
        try:
            os.symlink(source_rdkit_file, target_rdkit_file)
            print(f"Created symlink for RDKIT file")
        except Exception as e:
            print(f"Error creating symlink for RDKIT file: {e}")
    

Created symlink for CCD file
Created symlink for RDKIT file


In [2]:
import os, sys, json, tempfile, pathlib, subprocess, re, time
from   timeit import default_timer as timer
import numpy  as np
import pandas as pd
import torch
from   tqdm   import tqdm

# ── user flags ──────────────────────────────────────────────────────────
MODE        = "local"            #  <<<  "local"  or  "submit"
RUN_LOCAL   = False
RUN_KAGGLE  = not RUN_LOCAL

NUM_CONF=5
MAX_LENGTH=20000

assert torch.cuda.is_available(), "Need an NVIDIA GPU."
print("torch", torch.__version__, "| cuda:", torch.version.cuda,
      "| gpu:", torch.cuda.get_device_name(0))

# ── pip installs (done once) ────────────────────────────────────────────
# !pip install --no-deps protenix biopython ml-collections \
#                       biotite==1.0.1 rdkit

!pip install --no-index --no-deps --find-links=/kaggle/input/protenix-wheel/wheels protenix biopython ml-collections biotite==1.0.1 rdkit

# ── Protenix resource directory ────────────────────────────────────────
os.environ["USE_DEEPSPEED_EVO_ATTENTION"] = "false"

if RUN_LOCAL:
    ROOT_DIR = "/home/max/Documents/Protenix-KaggleRNA3D/af3-dev"
else:
    ROOT_DIR = "/kaggle/input/protenix-public/af3-dev"
        
os.environ["PROTENIX_DATA_ROOT_DIR"] = ROOT_DIR
print("PROTENIX_DATA_ROOT_DIR →", ROOT_DIR)

# ── CONFIDENCE CALCULATION FUNCTION ─────────────────────────────────────────
def compute_single_confidence_score(plddt_per_residue):
    """
    Compute a single confidence score from per-residue pLDDT scores.
    This is the STANDARD method used to ensure consistency.
    """
    if len(plddt_per_residue) == 0:
        return 0.0
    return float(np.mean(plddt_per_residue))

torch 2.5.1+cu124 | cuda: 12.4 | gpu: Tesla P100-PCIE-16GB
Looking in links: /kaggle/input/protenix-wheel/wheels
Processing /kaggle/input/protenix-wheel/wheels/protenix-0.4.6-py3-none-any.whl
Processing /kaggle/input/protenix-wheel/wheels/biopython-1.83-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/protenix-wheel/wheels/ml_collections-1.1.0-py3-none-any.whl
Processing /kaggle/input/protenix-wheel/wheels/biotite-1.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/protenix-wheel/wheels/rdkit-2024.9.6-cp311-cp311-manylinux_2_28_x86_64.whl
PROTENIX_DATA_ROOT_DIR → /kaggle/input/protenix-public/af3-dev


In [3]:
# SEQ_CSV = ("/home/max/Documents/Protenix-KaggleRNA3D/data/stanford-rna-3d-folding/"
#            f'{"validation" if MODE=="local" else "test"}_sequences.csv')
SEQ_CSV = "/kaggle/input/validation-sequences-clean-csv/validation_sequences_clean.csv"
df      = pd.read_csv(SEQ_CSV)

if MODE == "local":
    # LABEL_CSV  = "/home/max/Documents/Protenix-KaggleRNA3D/data/stanford-rna-3d-folding/validation_labels.csv"
    LABEL_CSV  = "/kaggle/input/validation-labels-clean-csv/validation_labels_clean.csv"
    label_df   = pd.read_csv(LABEL_CSV)
    label_df["target_id"] = label_df.ID.str.rsplit(pat="_", n=1).str[0]

# build input JSON --------------------------------------------------------
samples = [{"name":tid,
            "sequences":[{"rnaSequence":{"sequence":seq,"count":1}}]}
           for seq,tid in zip(df.sequence, df.target_id)]
json_path = tempfile.mktemp(prefix="protenix_inputs_", suffix=".json")
json.dump(samples, open(json_path,"w"))
print("json →", json_path)

json → /tmp/protenix_inputs_8q_1iu4i.json


In [4]:
# !cat /tmp/protenix_inputs_7x1cqjsu.json


In [5]:
# ! ln -s /kaggle/input/protenix/af3-dev/release_data /release_data
# ! ls /release_data

In [6]:
from configs.configs_base       import configs as cfg_base
from configs.configs_data       import data_configs
from configs.configs_inference  import inference_configs
from protenix.config.config     import parse_configs
from runner.inference           import InferenceRunner, update_inference_configs

ckpt_path = f"{ROOT_DIR}/release_model/model_v0.2.0.pt"

cfg_base["use_deepspeed_evo_attention"]     = False
cfg_base["model"]["N_cycle"]                = 10
cfg_base["sample_diffusion"]["N_step"]      = 200
cfg_base["sample_diffusion"]["N_sample"]    = 5          # 1 if VRAM is tight
inference_configs["load_checkpoint_path"]   = ckpt_path
inference_configs["dtype"]                  = "bf16"     # GPU friendly

cfg = { **cfg_base,
        **{"data": data_configs},
        **inference_configs,
        "input_json_path": json_path,
        "dump_dir": tempfile.mkdtemp(prefix="pred_out_") }

cfg = parse_configs(cfg, fill_required_with_null=True)
runner = InferenceRunner(cfg) 
print("model is", type(runner.model))

Try to find the ccd cache data in the code directory for inference.
train scheduler 16.0
inference scheduler 16.0
Diffusion Module has 16.0


/usr/local/lib/python3.11/dist-packages/runner/inference.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, self.device)


model is <class 'protenix.model.protenix.Protenix'>


In [7]:
from protenix.data.infer_data_pipeline import InferenceDataset
import time

ds = InferenceDataset(json_path, dump_dir=".", use_msa=False)
rows = []

for idx in tqdm(range(len(ds)), desc="Featurize → Predict"):
    start = time.time()
    data, atom_array, err = ds[idx]
    print(f"[{idx}] Dataset load time: {time.time() - start:.2f}s")
    
    tid  = data["sample_name"]
    seq  = df.loc[df.target_id == tid, "sequence"].values[0]
    
    # —— NEW LENGTH‐CUTOFF —— 
    if len(seq) > MAX_LENGTH:
        print(f"  ▶ Skipping {tid}: length {len(seq)} > {MAX_LENGTH}")
        # pad with zeros so row‐count stays correct
        for i, res in enumerate(seq, 1):
            rows.append([f"{tid}_{i}", res, i] + [0.0]* (NUM_CONF*3))
        continue
    # ————————————————

    if err:
        print("error:" + err)
        # your existing error‐padding
        for i, res in enumerate(seq, 1):
            rows.append([f"{tid}_{i}", res, i] + [0.0]* (NUM_CONF*3))
        continue

    # now safe to run Protenix on a sequence ≤ MAX_LENGTH
    start = time.time()
    runner.update_model_configs(update_inference_configs(cfg, int(data["N_token"])))
    print(f"[{idx}] Config update time: {time.time() - start:.2f}s")
    
    start = time.time()
    with torch.no_grad():
        prediction_output = runner.predict(data)
        coord = prediction_output["coordinate"]
        
        # Extract pLDDT scores
        plddt_scores = None
        for key in ["plddt", "confidence", "plDDT", "lddt"]:
            if key in prediction_output:
                plddt_scores = prediction_output[key]
                break
        
        if plddt_scores is None:
            print(f"Warning: No confidence scores found for {tid}")
            print(f"Available keys: {list(prediction_output.keys())}")
            plddt_scores = torch.zeros(coord.shape[0], coord.shape[1])
            
    print(f"[{idx}] Prediction time: {time.time() - start:.2f}s")

    c1_mask = data["input_feature_dict"]["atom_to_tokatom_idx"] == 12
    coord   = coord[:, c1_mask, :]                  # [N_sample, L, 3]
    # Extract pLDDT for C1' atoms only (to match coordinate extraction)
    if isinstance(plddt_scores, torch.Tensor):
        plddt_c1 = plddt_scores[:, c1_mask.cpu().numpy()]  # [N_sample, L_c1]
    else:
        plddt_c1 = torch.zeros(NUM_CONF, len(seq))

    # ensure exactly NUM_CONF samples for both coordinates and confidence
    while coord.shape[0] < NUM_CONF:
        coord = torch.cat([coord, coord[-1:]], dim=0)
        plddt_c1 = torch.cat([plddt_c1, plddt_c1[-1:]], dim=0)

    # Compute single confidence score per prediction
    confidence_scores = []
    for sample_idx in range(NUM_CONF):
        if isinstance(plddt_c1, torch.Tensor):
            plddt_per_residue = plddt_c1[sample_idx].cpu().numpy()
        else:
            plddt_per_residue = np.zeros(len(seq))
        
        sequence_confidence = compute_single_confidence_score(plddt_per_residue)
        confidence_scores.append(sequence_confidence)
    
    # Store confidence for this target
    confidence_data = {
        'target_id': tid,
        'sequence_length': len(seq),
        'confidence_1': confidence_scores[0],
        'confidence_2': confidence_scores[1],
        'confidence_3': confidence_scores[2],
        'confidence_4': confidence_scores[3],
        'confidence_5': confidence_scores[4]
    }
    if 'confidence_rows' not in globals():
        confidence_rows = []
    confidence_rows.append(confidence_data)

    for i, res in enumerate(seq, 1):
        triplets = coord[:, i-1, :].cpu().numpy().reshape(-1)
        rows.append([f"{tid}_{i}", res, i] + triplets.tolist() + confidence_scores)


coord_cols = [f"{ax}_{k}" for k in range(1,6) for ax in ("x","y","z")]
conf_cols = [f"confidence_{k}" for k in range(1,6)]
cols = (["ID", "resname", "resid"] + coord_cols + conf_cols)
sub  = pd.DataFrame(rows, columns=cols)
sub.to_csv("submission.csv", index=False)
print("submission.csv written — shape:", sub.shape)
sub.head()

Featurize → Predict:   0%|          | 0/94 [00:00<?, ?it/s]

[0] Dataset load time: 16.67s
[0] Config update time: 0.00s


/usr/local/lib/python3.11/dist-packages/protenix/openfold_local/model/primitives.py:238: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/usr/local/lib/python3.11/dist-packages/protenix/openfold_local/model/primitives.py:215: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/usr/local/lib/python3.11/dist-packages/protenix/openfold_local/model/primitives.py:280: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Featurize → Predict:   1%|          | 1/94 [02:10<3:21:41, 130.12s/it]

[0] Prediction time: 113.44s
[1] Dataset load time: 0.23s
[1] Config update time: 0.00s


Featurize → Predict:   2%|▏         | 2/94 [03:03<2:09:58, 84.77s/it] 

...

[92] Prediction time: 51.08s
[93] Dataset load time: 0.44s
[93] Config update time: 0.00s


Featurize → Predict: 100%|██████████| 94/94 [3:15:32<00:00, 124.81s/it]

[93] Prediction time: 76.66s


submission.csv written — shape: (13782, 23)


,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,y_4,z_4,x_5,y_5,z_5,confidence_1,confidence_2,confidence_3,confidence_4,confidence_5
0,9L5R_2_1,A,1,-23.133446,5.648046,-55.872604,24.893173,15.280736,39.966980,5.991918,...,-9.331330,64.695862,-48.705055,13.416087,29.238045,0.062999,0.048493,0.090558,0.110703,0.090135
1,9L5R_2_2,G,2,-24.690014,7.824708,-51.199677,19.621548,17.113182,38.998108,5.593056,...,-12.778950,60.663078,-43.725693,15.635481,29.560118,0.062999,0.048493,0.090558,0.110703,0.090135
2,9L5R_2_3,C,3,-27.746859,7.639439,-46.665176,15.809040,18.199286,35.121109,2.458481,...,-15.455084,55.955589,-38.693649,15.851530,31.901075,0.062999,0.048493,0.090558,0.110703,0.090135
3,9L5R_2_4,U,4,-30.402573,4.543214,-43.304604,14.658719,16.589390,30.191492,-2.466850,...,-15.825209,51.451309,-34.772202,13.331332,34.730221,0.062999,0.048493,0.090558,0.110703,0.090135
4,9L5R_2_5,C,5,-31.527815,-0.321229,-41.511032,15.135366,11.992863,27.347424,-7.385670,...,-13.567739,47.877548,-32.685989,8.666122,36.402740,0.062999,0.048493,0.090558,0.110703,0.090135


In [8]:
import shutil
import os

# Copy USalign to working directory and make it executable
shutil.copy2("/kaggle/input/usalign/USalign", "/kaggle/working/USalign")
os.chmod("/kaggle/working/USalign", 0o755)

print("USalign copied to /kaggle/working/ and made executable")

USalign copied to /kaggle/working/ and made executable


In [9]:
import os
import re
import numpy as np
import pandas as pd

def parse_tmscore_output(output):
    """Parse TM-score from USalign output with detailed debugging"""
    print(f"DEBUG: Raw USalign output:")
    print(f"'{output}'")
    print(f"DEBUG: Output length: {len(output)}")
    
    if not output.strip():
        print("Warning: Empty output from USalign")
        return 0.0
    
    # Look for all TM-score patterns
    tm_score_matches = re.findall(r'TM-score=\s*([\d.]+)', output)
    print(f"DEBUG: Found TM-score matches: {tm_score_matches}")
    
    if len(tm_score_matches) == 0:
        print("Warning: No TM-score found in output")
        return 0.0
    elif len(tm_score_matches) == 1:
        print("Warning: Only one TM-score found, using it")
        return float(tm_score_matches[0])
    else:
        print(f"Found {len(tm_score_matches)} TM-scores, using the second one")
        return float(tm_score_matches[1])

def write_target_line(
    atom_name, atom_serial, residue_name, chain_id, residue_num,
    x_coord, y_coord, z_coord, occupancy=1.0, b_factor=0.0, atom_type='P'
) -> str:
    return (
        f'ATOM  {atom_serial:>5d}  {atom_name:<5s} {residue_name:<3s} '
        f'{residue_num:>3d}    {x_coord:>8.3f}{y_coord:>8.3f}'
        f'{z_coord:>8.3f}{occupancy:>6.2f}{b_factor:>6.2f}           {atom_type}\n'
    )

def write2pdb(df: pd.DataFrame, xyz_id: int, target_path: str) -> int:
    resolved_cnt = 0
    with open(target_path, 'w') as f:
        for _, row in df.iterrows():
            x = row[f'x_{xyz_id}']; y = row[f'y_{xyz_id}']; z = row[f'z_{xyz_id}']
            if x > -1e17 and y > -1e17 and z > -1e17:
                resolved_cnt += 1
                f.write(write_target_line(
                    atom_name="C1'", atom_serial=int(row['resid']),
                    residue_name=row['resname'], chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x, y_coord=y, z_coord=z, atom_type='C'
                ))
    return resolved_cnt

def test_usalign():
    """Test if USalign is working properly"""
    usalign_path = "/kaggle/working/USalign"
    
    # Check if file exists
    if not os.path.exists(usalign_path):
        print(f"ERROR: USalign not found at {usalign_path}")
        return False
    
    # Check if it's executable
    if not os.access(usalign_path, os.X_OK):
        print(f"ERROR: USalign at {usalign_path} is not executable")
        print("Trying to make it executable...")
        os.chmod(usalign_path, 0o755)
    
    # Test basic execution
    try:
        test_output = os.popen(f'{usalign_path} 2>&1').read()
        print(f"USalign test output: {test_output[:200]}...")
        return True
    except Exception as e:
        print(f"ERROR testing USalign: {e}")
        return False

def score_and_report_debug(solution: pd.DataFrame, submission: pd.DataFrame):
    """Scoring function with extensive debugging"""
    print("=== Starting scoring with debug output ===")
    
    # Test USalign first
    if not test_usalign():
        print("USalign test failed, cannot proceed with scoring")
        return {}, 0.0
    
    # extract target_id
    solution['target_id'] = solution['ID'].str.split('_').str[0]
    submission['target_id'] = submission['ID'].str.split('_').str[0]

    native_idxs = sorted(int(c.split('_')[1])
                         for c in solution.columns if c.startswith('x_'))
    print(f"Native structure indices: {native_idxs}")

    usalign = "/kaggle/working/USalign"
    per_target = {}

    # Test with just the first target for debugging
    target_ids = solution['target_id'].unique()
    print(f"Found {len(target_ids)} targets, testing first one for debugging...")
    
    for target_idx, (tid, grp_nat) in enumerate(solution.groupby('target_id')):
        print(f"\n=== Processing target {tid} ({target_idx+1}/{len(target_ids)}) ===")
        grp_pred = submission[submission['target_id'] == tid]
        
        print(f"Native group shape: {grp_nat.shape}")
        print(f"Predicted group shape: {grp_pred.shape}")
        
        best_of_five = []

        for pred_cnt in range(1, 6):
            print(f"\n--- Testing prediction {pred_cnt} ---")
            best_for_this_pred = 0.0
            
            for nat_cnt in native_idxs:
                print(f"Comparing prediction {pred_cnt} vs native {nat_cnt}")
                
                n_nat = write2pdb(grp_nat, nat_cnt, 'native.pdb')
                n_pred = write2pdb(grp_pred, pred_cnt, 'predicted.pdb')
                
                print(f"Native atoms written: {n_nat}, Predicted atoms written: {n_pred}")
                
                if n_nat > 0 and n_pred > 0:
                    cmd = f'{usalign} predicted.pdb native.pdb -atom " C1\'"'
                    print(f"Running command: {cmd}")
                    
                    try:
                        out = os.popen(cmd).read()
                        score = parse_tmscore_output(out)
                        print(f"TM-score: {score}")
                        best_for_this_pred = max(best_for_this_pred, score)
                    except Exception as e:
                        print(f"Error running USalign: {e}")
                        continue
                else:
                    print("Skipping due to empty structures")
            
            best_of_five.append(best_for_this_pred)
            print(f"Best score for prediction {pred_cnt}: {best_for_this_pred}")

        per_target[tid] = best_of_five
        print(f"{tid}: TM-scores per model = {best_of_five}, best = {max(best_of_five):.4f}")
        
        # Only process first target for debugging, remove this break for full scoring
        if target_idx == 0:
            print("=== Debug mode: stopping after first target ===")
            break

    overall = np.mean([max(v) for v in per_target.values()]) if per_target else 0.0
    print(f"\n>>> mean best-of-5 TM-score = {overall:.4f}")
    return per_target, overall

# Quick function to check PDB files
def check_pdb_files():
    """Check if PDB files are being created correctly"""
    for filename in ['native.pdb', 'predicted.pdb']:
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                content = f.read()
                print(f"\n=== {filename} content (first 500 chars) ===")
                print(content[:500])
                print(f"=== {filename} total lines: {len(content.splitlines())} ===")
        else:
            print(f"{filename} does not exist")

# Main execution
if __name__ == "__main__":
    solution = pd.read_csv(
        "/kaggle/input/validation-labels-clean-csv/validation_labels_clean.csv"
    )
    submission = pd.read_csv("submission.csv")

    print("Solution columns:", solution.columns.tolist())
    print("Submission columns:", submission.columns.tolist())
    print("Solution shape:", solution.shape)
    print("Submission shape:", submission.shape)

    # Run debug scoring
    per_target_scores, mean_tm = score_and_report_debug(solution, submission)
    
    # Check PDB files after scoring
    check_pdb_files()

Solution columns: ['ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1']
Submission columns: ['ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2', 'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5', 'z_5', 'confidence_1', 'confidence_2', 'confidence_3', 'confidence_4', 'confidence_5']
Solution shape: (13782, 6)
Submission shape: (13782, 23)
=== Starting scoring with debug output ===
USalign test output: 
 ********************************************************************
 * US-align (Version 20241108)                                      *
 * Universal Structure Alignment of Proteins and Nucleic Ac...
Native structure indices: [1]
Found 83 targets, testing first one for debugging...

=== Processing target 8K85 (1/83) ===
Native group shape: (56, 7)
Predicted group shape: (56, 24)

--- Testing prediction 1 ---
Comparing prediction 1 vs native 1
Native atoms written: 56, Predicted atoms written: 56
Running command: /kaggle/working/USalign predicted.pdb native.pdb -atom " C1

In [10]:
submission.to_csv("/kaggle/working/protenix_submission_with_confidence.csv", index=False)